---
title: Core
subtitle: Core data structures and algorithms
---


In [ ]:
# | default_exp core
# | export
from pydantic import ConfigDict
from pydantic.dataclasses import dataclass
from datetime import datetime

## Variable

In [ ]:

# | export
@dataclass(config=ConfigDict(extra='allow'))
class Variable:
    name: str = None
    description: str = None
    unit: str = None
    timerange: list[datetime] = None
    
@dataclass
class Variables:
    timerange: list[datetime] = None
    variables: dict[str, Variable] = None
    def add_variable(self, variable: Variable):
        self.variables[variable.name] = variable

## Dataset

In [ ]:
# | export
@dataclass
class Dataset(Variables):
    name: str = None

## Instrument and Mission

In [ ]:
# | export
@dataclass
class Instrument:
    name: str
    datasets: dict[str, Dataset] = None

    def add_dataset(self, dataset: Dataset):
        self.datasets[dataset.name] = dataset


@dataclass
class Mission:
    name: str
    """Name of the mission"""
    instruments: dict[str, Instrument] = None
    datasets: dict[str, Dataset] = None

    def add_instrument(self, instrument: Instrument):
        self.instruments[instrument.name] = instrument

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()